# Vision - Walk datasets

## Python imports

In [1]:
import csv
import cv2
import os.path
import pandas as pd
import re
import VideoCapture

## Parameters

In [2]:
DEVICE = 'Pixel'

DPARAM = {
    'Pixel' : {
        'video': 'raw_data/Pixel/Vision/1228187371660504_teste_final.mp4',
        'walk_times': 'raw_data/Pixel/walk-times.tsv'
    },
    'Redmi': {
        'video': 'raw_data/Redmi/Vision/97082214722162_teste_final.3gp',
        'walk_times': 'raw_data/Redmi/walk-times.tsv'        
    }
}
TIME_GRANULARITY = 1_000_000_000
MERGE_ATRIO_LABELS = False

## Read walk data

In [3]:
time_re = re.compile('^(\d+):(\d+)$')
times = []
with open(DPARAM[DEVICE]['walk_times'], newline='') as csv_input:
    data = csv.reader(csv_input, delimiter='\t')
    for row in data:
      (begin,end,label) = row[1:4]
      # print(begin,end,label)
      m = time_re.match(begin)
      begin = int(m.group(1)) * 60 + int(m.group(2)) 
      m = time_re.match(end)
      end = int(m.group(1)) * 60 + int(m.group(2))
      begin *= 1_000_000_000 / TIME_GRANULARITY
      end *= 1_000_000_000 / TIME_GRANULARITY
      if MERGE_ATRIO_LABELS and "AT_" in label:
        label = "A"
      times.append((begin,end,label))
WALK_TIMES_DATA = pd.DataFrame({'begin': [t[0] for t in times],
                  'end': [t[1] for t in times],
                  'label': [t[2] for t in times]})
WALK_TIMES_DATA

,begin,end,label
0,0.0,19.0,SS
1,20.0,40.0,TMA
2,41.0,54.0,DC
3,55.0,71.0,DG
4,72.0,91.0,TS
5,92.0,107.0,AH
6,108.0,154.0,SA
7,155.0,186.0,DF
8,187.0,204.0,CN
9,204.0,216.0,SN


## Map labels 

In [4]:
time_2_label = []
for r in WALK_TIMES_DATA.itertuples():
    t_begin = int(r.begin)
    t_end = int(r.end)
    for _ in range(t_end - t_begin+1):
        time_2_label.append(r.label)
len(time_2_label)

375

In [5]:
vc = VideoCapture.VideoInput(DPARAM[DEVICE]['video'])
next_time = 0
time_limit = len(time_2_label) - 1
print('Time:', vc.frame_count / vc.fps)
dataset_tsv_file = open('datasets/vision/PW.tsv' if DEVICE == 'Pixel' else 'datasets/vision/RW.tsv','w')
print('label', 'set', 'time', 'source', 'image', 'video', sep='\t', file=dataset_tsv_file)
while next_time < time_limit:
    _, frame = vc.next_frame()
    label = time_2_label[next_time]
    directory = 'datasets/vision/images/walk/%s/%s' % (DEVICE,label)
    os.makedirs(directory, exist_ok=True)
    image_file = directory + "/%04d_%05d.jpeg" % (next_time, vc.current_frame)
    cv2.imwrite(image_file, frame)
    print(label, 'test', next_time,  DEVICE, image_file, vc.video_file, sep='\t', file=dataset_tsv_file)
    print(next_time, label, image_file, vc.current_frame, sep='\t')
    next_time += 1
    while vc.current_time < next_time:
        _ = vc.next_frame()
dataset_tsv_file.close()

Time: 382.08703333333335
0	SS	datasets/vision/images/walk/Pixel/SS/0000_00000.jpeg	0
1	SS	datasets/vision/images/walk/Pixel/SS/0001_00031.jpeg	31
2	SS	datasets/vision/images/walk/Pixel/SS/0002_00061.jpeg	61
3	SS	datasets/vision/images/walk/Pixel/SS/0003_00091.jpeg	91
4	SS	datasets/vision/images/walk/Pixel/SS/0004_00121.jpeg	121
5	SS	datasets/vision/images/walk/Pixel/SS/0005_00151.jpeg	151
6	SS	datasets/vision/images/walk/Pixel/SS/0006_00181.jpeg	181
7	SS	datasets/vision/images/walk/Pixel/SS/0007_00211.jpeg	211
8	SS	datasets/vision/images/walk/Pixel/SS/0008_00241.jpeg	241
9	SS	datasets/vision/images/walk/Pixel/SS/0009_00271.jpeg	271
10	SS	datasets/vision/images/walk/Pixel/SS/0010_00301.jpeg	301
11	SS	datasets/vision/images/walk/Pixel/SS/0011_00331.jpeg	331
12	SS	datasets/vision/images/walk/Pixel/SS/0012_00361.jpeg	361
13	SS	datasets/vision/images/walk/Pixel/SS/0013_00391.jpeg	391
14	SS	datasets/vision/images/walk/Pixel/SS/0014_00421.jpeg	421
15	SS	datasets/vision/images/walk/Pixel/SS/00

131	SA	datasets/vision/images/walk/Pixel/SA/0131_03932.jpeg	3932
132	SA	datasets/vision/images/walk/Pixel/SA/0132_03962.jpeg	3962
133	SA	datasets/vision/images/walk/Pixel/SA/0133_03992.jpeg	3992
134	SA	datasets/vision/images/walk/Pixel/SA/0134_04022.jpeg	4022
135	SA	datasets/vision/images/walk/Pixel/SA/0135_04052.jpeg	4052
136	SA	datasets/vision/images/walk/Pixel/SA/0136_04082.jpeg	4082
137	SA	datasets/vision/images/walk/Pixel/SA/0137_04112.jpeg	4112
138	SA	datasets/vision/images/walk/Pixel/SA/0138_04142.jpeg	4142
139	SA	datasets/vision/images/walk/Pixel/SA/0139_04172.jpeg	4172
140	SA	datasets/vision/images/walk/Pixel/SA/0140_04202.jpeg	4202
141	SA	datasets/vision/images/walk/Pixel/SA/0141_04232.jpeg	4232
142	SA	datasets/vision/images/walk/Pixel/SA/0142_04263.jpeg	4263
143	SA	datasets/vision/images/walk/Pixel/SA/0143_04293.jpeg	4293
144	SA	datasets/vision/images/walk/Pixel/SA/0144_04323.jpeg	4323
145	SA	datasets/vision/images/walk/Pixel/SA/0145_04353.jpeg	4353
146	SA	datasets/vision/im

258	HA	datasets/vision/images/walk/Pixel/HA/0258_07744.jpeg	7744
259	HA	datasets/vision/images/walk/Pixel/HA/0259_07774.jpeg	7774
260	HA	datasets/vision/images/walk/Pixel/HA/0260_07804.jpeg	7804
261	HA	datasets/vision/images/walk/Pixel/HA/0261_07834.jpeg	7834
262	HA	datasets/vision/images/walk/Pixel/HA/0262_07864.jpeg	7864
263	HA	datasets/vision/images/walk/Pixel/HA/0263_07894.jpeg	7894
264	HA	datasets/vision/images/walk/Pixel/HA/0264_07924.jpeg	7924
265	HA	datasets/vision/images/walk/Pixel/HA/0265_07954.jpeg	7954
266	HA	datasets/vision/images/walk/Pixel/HA/0266_07984.jpeg	7984
267	HA	datasets/vision/images/walk/Pixel/HA/0267_08014.jpeg	8014
268	HA	datasets/vision/images/walk/Pixel/HA/0268_08044.jpeg	8044
269	AT_I1	datasets/vision/images/walk/Pixel/AT_I1/0269_08074.jpeg	8074
270	AT_I1	datasets/vision/images/walk/Pixel/AT_I1/0270_08104.jpeg	8104
271	AT_I1	datasets/vision/images/walk/Pixel/AT_I1/0271_08134.jpeg	8134
272	AT_I1	datasets/vision/images/walk/Pixel/AT_I1/0272_08164.jpeg	8164
2